In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tools import whiten, adap_whitening, adap_whitening_2
from sklearn import svm, metrics
import sklearn
import pickle
from scipy.ndimage import gaussian_filter1d
from tools import load, split, estimate_derivative, plot_two_intervals

In [ ]:
def train_st(sensor_data, sequence, times_sec, sequence_sec,
          n_hd=10000, n_out=3, k=10, n_pot=12., w_teacher=1., t_training_delay=2.,
          normalized=False, whitened=False, uniformW=False,):

    if normalized:
        sensor_data_norm = (sensor_data - np.mean(sensor_data, axis=0))/ np.std(sensor_data, axis=0)
    else:
        sensor_data_norm = sensor_data
    if whitened:
        x_dense = adap_whitening_2(sensor_data_norm)
    else:
        x_dense = sensor_data_norm

    n_dense = x_dense.shape[1]

    labels = np.zeros_like(times_sec)
    for i, t in enumerate(sequence_sec):
        try:
            flag = (times_sec > sequence_sec[i] + t_training_delay) & (times_sec < sequence_sec[i+1])
        except IndexError:
            flag = (times_sec > sequence_sec[i] + t_training_delay)
        labels[flag] = int(sequence[i][1])

    if uniformW:
        W_hd = np.random.uniform(high=1/np.sqrt(n_dense), size=(n_hd, n_dense))  #Test random sparse weights
    else:
        W_hd = np.random.binomial(n=1, p=0.05, size=(n_hd, n_dense))  #Test random sparse weights

    x_hd = x_dense @ W_hd.T
    z_hd = np.where(np.argsort(x_hd)<k, 1., 0)
    W_out = np.zeros((n_out, n_hd))
    W = np.zeros((n_out, n_hd))
    z_out_train = np.zeros((z_hd.shape[0],  n_out))

    for i, row in enumerate(z_hd):
        teacher = np.zeros((n_out,))
        if labels[i] != 0:
            teacher[int(labels[i]-1)] = w_teacher
        out = row @ W_out.T + teacher
        z_out_train[i] = out
        dW = (1./n_pot)*(np.atleast_2d(out).T @ np.atleast_2d(row))
        W += dW
        W_out = np.where(W>=1., 1./k, 0.)

    return W_hd, W_out

def test_st(sensor_data, sequence, times_sec, sequence_sec,
         W_hd, W_out,
         n_hd=10000, n_out=3, k=10, integration_delay=0.,
         normalized=False, whitened=False):

    if normalized:
        sensor_data_norm = (sensor_data - np.mean(sensor_data, axis=0))/ np.std(sensor_data, axis=0)
    else:
        sensor_data_norm = sensor_data
    if whitened:
        x_dense = adap_whitening_2(sensor_data_norm)
    else:
        x_dense = sensor_data_norm

    x_hd = x_dense @ W_hd.T
    z_hd = np.where(np.argsort(x_hd)<k, 1., 0)
    z_out = np.zeros((z_hd.shape[0],  n_out))
    for i, row in enumerate(z_hd):
        out = row @ W_out.T
        z_out[i] = out

    z_pred = np.zeros_like(sequence_sec)
    z_true = np.zeros_like(sequence_sec)
    for i, t in enumerate(sequence_sec):
        try:
            flag = (times_sec > sequence_sec[i] + integration_delay) & (times_sec < sequence_sec[i+1])
        except IndexError:
            flag = (times_sec > sequence_sec[i] + integration_delay)
        z_pred[i] = np.argsort(np.sum(z_out[flag], axis=0))[-1] + 1
        z_true[i] = sequence[i][1]
    test_acc = sklearn.metrics.accuracy_score(z_true, z_pred)

    return test_acc